In [1]:
#Residual Returns (beta adjusted returns)

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime 
from pandas_datareader import data
import statistics 

In [2]:
pd.set_option('display.precision', 4)

In [3]:
def get_start_dt(dt_, offset_days_=260):
    offset = pd.tseries.offsets.BusinessDay(n=offset_days_)
    out_dt = dt_ - offset
    return out_dt.date()

In [4]:
beta = lambda s_ret, indx_ret, range_end_dt: np.cov(s_ret.loc[:range_end_dt], indx_ret.loc[:range_end_dt])[0, 1] / \
                                             np.cov(s_ret.loc[:range_end_dt], indx_ret.loc[:range_end_dt])[1, 1]


In [5]:
res_returns = lambda tkr: ret_df[tkr].subtract(risk_free_rate) - betas[tkr] * (ret_df['SPY'].subtract(risk_free_rate))

In [6]:
def compute_residual_rets (tkr_list_):
    r = [res_returns(tkr) for tkr in tkr_list_]
    rd = pd.DataFrame(r).T
    rd.columns = tkr_list_
    return rd

In [7]:
tickers_file = 'Water Related Utilities tickers California Drought.xlsx'
ticker = pd.read_excel(tickers_file, index_col='ticker')
ticker

,name
ticker,
CGW,Invesco S&P Global Water Index ETF
PHO,Invesco Water Resources ETF


In [8]:
tkr_list_HL = list(ticker.index)

In [9]:
tkr_list_HL

['CGW', 'PHO']

stocks = list(ticker.reset_index().T.to_dict().values())
stocks

#stocks = [
    {
        'ticker': 'XOM',
        'name': 'Exxon'
    },
    {
        'ticker': 'CVX',
        'name': 'Chevron'
    },
    {
        'ticker': 'OXY',
        'name': 'Occidental'
    },
    {
        'ticker': 'COP',
        'name': 'ConocoPhillips'
    },
    {
        'ticker': 'EOG',
        'name': 'EOG Resources Group'
    },
    {
        'ticker': 'PXD',
        'name': 'Pioneer Natural Resources'
    },
     {
        'ticker': 'MPC',
        'name': 'Marathon Petroleum'
    },
    {
        'ticker': 'SLB',
        'name': 'Schlumberger Limited'
    },
    {
        'ticker': 'VLO',
        'name': 'Valero Energy Corporation'
    },
    {
        'ticker': 'PSX',
        'name': 'Philips 66'
    },
    {
        'ticker': 'WMB',
        'name': 'The Williams Companies Inc'
    },
    {
        'ticker': 'DVN',
        'name': 'Devon Energy'
    },
    {
        'ticker': 'KMI',
        'name': 'Kinder Morgan Inc'
    },
    {
        'ticker': 'HES',
        'name': 'HESS Corporation'
    },
    {
        'ticker': 'OKE',
        'name': 'ONEOK INC'
    },
     {
        'ticker': 'HAL',
        'name': 'Halliburton Company'
    },
    {
        'ticker': 'CTRA',
        'name': 'Coterra Energy'
    },
    {
        'ticker': 'BKR',
        'name': 'Baker Hughes Company'
    },
    {
        'ticker': 'FANG',
        'name': 'Diamondback Energy Inc'
    },
    {
        'ticker': 'MRO',
        'name': 'Marathon Oil Corporation'
    },
    {
        'ticker': 'APA',
        'name': 'APA Corporation'
    },
    {
        'ticker': 'EQT',
        'name': 'EQT Corp'
    }
]

stocks_df = pd.DataFrame(stocks)
stocks_df.set_index('ticker', inplace=True)
tkr_list = list(stocks_df.index)

tkr_list

In [10]:
crisis_dt = datetime(2022, 8, 1)
end_dt = datetime(2022, 9, 14)
start_dt = get_start_dt(end_dt, offset_days_=260)

In [11]:
cl_df = data.DataReader(['SPY'] + tkr_list_HL, start=start_dt, end=end_dt, data_source='yahoo')['Adj Close']

cl_df = data.DataReader(['SPY'] + ['EOG', 'COP'], start=start_dt, end=end_dt, data_source='yahoo')['Adj Close']

In [12]:
ret_df = cl_df.div(cl_df.shift(1)) - 1.0 
ret_df.dropna(inplace=True)
index_return = ret_df['SPY']
index_var = np.var(ret_df['SPY'])

In [13]:
betas = ret_df[tkr_list_HL].apply(beta, axis=0, indx_ret= ret_df['SPY'], range_end_dt=crisis_dt)

In [14]:
treasury_3_mo = 2.943/100
risk_free_rate = treasury_3_mo/90

In [15]:
residual_return_df = compute_residual_rets(tkr_list_HL)

In [16]:
residual_return_df

,CGW,PHO
Date,,
2021-09-16,-4.1761e-03,-0.0093
2021-09-17,-7.9936e-03,-0.0017
2021-09-20,3.1318e-03,0.0102
2021-09-21,1.2094e-03,-0.0027
2021-09-22,-6.1873e-05,-0.0015
...,...,...
2022-09-08,-2.4757e-03,0.0022
2022-09-09,4.3520e-03,-0.0037
2022-09-12,4.7159e-03,-0.0014


In [17]:
max_by_columns = residual_return_df.max()
max_by_rows = residual_return_df.max(axis=1)

df_max = max(max_by_columns)
print("Max element based on the list of columns: ", df_max)

df_max2 = max(max_by_rows)
print("Max element based on the list of rows: ", df_max2)


Max element based on the list of columns:  0.017707147798492362
Max element based on the list of rows:  0.017707147798492362


In [18]:
median_by_columns = residual_return_df.median()
median_by_rows = residual_return_df.median(axis=1)

df_median = statistics.median(median_by_columns)
print("Median element based on the list of columns: ", df_median)

df_median2 = statistics.median(median_by_rows)
print("Median element based on the list of rows: ", df_median2)


Median element based on the list of columns:  -0.00048381633079205367
Median element based on the list of rows:  -0.0004179477424445066


In [19]:
min_by_columns = residual_return_df.min()
min_by_rows = residual_return_df.min(axis=1)

df_min = min(min_by_columns)
print("Minimum element based on the list of columns: ", df_min)

df_min2 = min(min_by_rows)
print("Minimum  element based on the list of rows: ", df_min2)


Minimum element based on the list of columns:  -0.02218757033338487
Minimum  element based on the list of rows:  -0.02218757033338487
